<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Human-numbers" data-toc-modified-id="Human-numbers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Human numbers</a></span><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Single-fully-connected-model" data-toc-modified-id="Single-fully-connected-model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Single fully connected model</a></span></li><li><span><a href="#Same-thing-with-a-loop" data-toc-modified-id="Same-thing-with-a-loop-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Same thing with a loop</a></span></li><li><span><a href="#Multi-fully-connected-model" data-toc-modified-id="Multi-fully-connected-model-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Multi fully connected model</a></span></li><li><span><a href="#Maintain-state" data-toc-modified-id="Maintain-state-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Maintain state</a></span></li><li><span><a href="#nn.RNN" data-toc-modified-id="nn.RNN-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>nn.RNN</a></span></li><li><span><a href="#2-layer-GRU" data-toc-modified-id="2-layer-GRU-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>2-layer GRU</a></span></li><li><span><a href="#fin" data-toc-modified-id="fin-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>fin</a></span></li></ul></li></ul></div>

# Human numbers

RNNs starts 1:58:55: https://www.youtube.com/watch?v=9shn3Oz7Ptw

In [1]:
from fastai import *
from fastai.text import *

In [2]:
bs=64

## Data

In [3]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/home/jupyter/.fastai/data/human_numbers/valid.txt'),
 PosixPath('/home/jupyter/.fastai/data/human_numbers/train.txt')]

In [4]:
def readnums(d): return [', '.join(o.strip() for o in open(path/d).readlines())]

In [5]:
train_txt = readnums('train.txt'); train_txt[0][:80]

'one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt'

In [6]:
valid_txt = readnums('valid.txt'); valid_txt[0][-80:]

' nine thousand nine hundred ninety eight, nine thousand nine hundred ninety nine'

In [7]:
## just one document
train = TextList(train_txt, path=path) ## 1 - 8000
valid = TextList(valid_txt, path=path) ## 8000 onwards

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs)

In [13]:
train[0].text[:80] ## index 0 because we only have one document
## xx is fast.ai token bos beginning of stream (start document)

'xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleve'

In [9]:
len(data.valid_ds[0][0].data) ## 13 words or punctuation marks

13017

In [10]:
data.bptt, len(data.valid_dl) ##bptt = back prop through time --> 64 equally sized (roughly) segments
##, that is not the same as length 64, here e.g. 70 = bptt (sequence length)

(70, 3)

In [11]:
13017/70/bs ## bs was 64
## around 3 batches

2.905580357142857

In [17]:
## 3 batches
it = iter(data.valid_dl)
x1,y1 = next(it)
x2,y2 = next(it)
x3,y3 = next(it)
it.close()

In [18]:
x1.numel()+x2.numel()+x3.numel()

12928

In [19]:
x1.shape,y1.shape
## size here is different than 70 (see above) because dataloader slightly randmoizes bptt to give more shuffling

(torch.Size([78, 64]), torch.Size([78, 64]))

In [20]:
x2.shape,y2.shape

(torch.Size([63, 64]), torch.Size([63, 64]))

In [21]:
x1[:,0]

tensor([ 2, 18, 10, 11,  8, 18, 10, 12,  8, 18, 10, 13,  8, 18, 10, 14,  8, 18,
        10, 15,  8, 18, 10, 16,  8, 18, 10, 17,  8, 18, 10, 18,  8, 18, 10, 19,
         8, 18, 10, 28,  8, 18, 10, 29,  8, 18, 10, 30,  8, 18, 10, 31,  8, 18,
        10, 32,  8, 18, 10, 33,  8, 18, 10, 34,  8, 18, 10, 35,  8, 18, 10, 36,
         8, 18, 10, 37,  8, 18], device='cuda:0')

In [23]:
## this is offset by 1 from x1[:,0], so after every word we predict the next word
y1[:,0]

tensor([18, 10, 11,  8, 18, 10, 12,  8, 18, 10, 13,  8, 18, 10, 14,  8, 18, 10,
        15,  8, 18, 10, 16,  8, 18, 10, 17,  8, 18, 10, 18,  8, 18, 10, 19,  8,
        18, 10, 28,  8, 18, 10, 29,  8, 18, 10, 30,  8, 18, 10, 31,  8, 18, 10,
        32,  8, 18, 10, 33,  8, 18, 10, 34,  8, 18, 10, 35,  8, 18, 10, 36,  8,
        18, 10, 37,  8, 18, 10], device='cuda:0')

In [24]:
v = data.valid_ds.vocab

Result from below: every mini batch joins up with the previous mini batch

In [25]:
## same as above, just as text
v.textify(x1[:,0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand eighteen , eight thousand nineteen , eight'

In [26]:
## same as above, just as text
v.textify(y1[:,0])

'eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand eighteen , eight thousand nineteen , eight thousand'

In [27]:
v.textify(x2[:,0])

'thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two , eight'

In [28]:
v.textify(x3[:,0])

'thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight thousand thirty seven , eight thousand thirty eight , eight thousand thirty nine , eight thousand forty , eight thousand forty one , eight thousand forty two , eight thousand forty three , eight thousand forty four , eight thousand forty'

In [29]:
v.textify(x1[:,1])

', eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty one'

In [30]:
v.textify(x2[:,1])

', eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand seventy'

In [31]:
v.textify(x3[:,1])

'four , eight thousand seventy five , eight thousand seventy six , eight thousand seventy seven , eight thousand seventy eight , eight thousand seventy nine , eight thousand eighty , eight thousand eighty one , eight thousand eighty two , eight thousand eighty three , eight thousand eighty four , eight thousand eighty five , eight thousand eighty six ,'

In [32]:
v.textify(x3[:,-1])

'nine hundred eighty seven , nine thousand nine hundred eighty eight , nine thousand nine hundred eighty nine , nine thousand nine hundred ninety , nine thousand nine hundred ninety one , nine thousand nine hundred ninety two , nine thousand nine hundred ninety three , nine thousand nine hundred ninety four , nine thousand nine hundred ninety five , nine'

In [33]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight"
1,", eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand"
2,"thousand eighty seven , eight thousand eighty eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine"
3,"thousand one hundred twenty three , eight thousand one hundred twenty four , eight thousand one hundred twenty five , eight thousand one hundred twenty six , eight thousand one hundred twenty seven , eight thousand one hundred twenty eight , eight thousand one hundred twenty nine , eight thousand one hundred thirty , eight thousand one hundred thirty one , eight"
4,"fifty two , eight thousand one hundred fifty three , eight thousand one hundred fifty four , eight thousand one hundred fifty five , eight thousand one hundred fifty six , eight thousand one hundred fifty seven , eight thousand one hundred fifty eight , eight thousand one hundred fifty nine , eight thousand one hundred sixty , eight thousand one hundred"


## Single fully connected model

In [34]:
data = src.databunch(bs=bs, bptt=3, max_len=0, p_bptt=1.)

In [35]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([3, 64]), torch.Size([3, 64]))

In [36]:
nv = len(v.itos); nv

38

In [37]:
nh=64

In [38]:
def loss4(input,target): return F.cross_entropy(input, target[-1]) ## target -1 : only last word of sequence
## below we compare every word in x with every word in y
def acc4 (input,target): return accuracy(input, target[-1])

In [62]:
## 2:09:30 https://www.youtube.com/watch?v=9shn3Oz7Ptw
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = self.bn(F.relu(self.i_h(x[0])))
        if x.shape[0]>1:
            h += self.i_h(x[1]) ## add word 1
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[0]>2:
            h += self.i_h(x[2]) ## add word 2
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [40]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

In [41]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4
1,3.572540,3.544832,0.056157
2,3.158949,3.176489,0.412129
3,2.635209,2.712034,0.452197
4,2.300514,2.425039,0.457843
5,2.165478,2.319739,0.457611
6,2.137440,2.304482,0.457611


## Same thing with a loop

In [63]:
## 2:11:10 https://www.youtube.com/watch?v=9shn3Oz7Ptw
## this is an RNN
## more general than above, works for any arbitrary length
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[1], nh).to(device=x.device)
        for xi in x:
            h += self.i_h(xi)
            h  = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [43]:
learn = Learner(data, Model1(), loss_func=loss4, metrics=acc4)

In [44]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4
1,3.537536,3.483274,0.127939
2,2.980145,2.932804,0.443843
3,2.411354,2.463387,0.468518
4,2.119241,2.253887,0.463567
5,2.011737,2.190600,0.461711
6,1.989772,2.181992,0.455446


## Multi fully connected model

In [45]:
data = src.databunch(bs=bs, bptt=20) ## bptt upped to 20

In [46]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([45, 64]), torch.Size([45, 64]))

In [47]:
## we predict here after every loop
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        ## reset state to zero
        ## instead we should keep it
        h = torch.zeros(x.shape[1], nh).to(device=x.device)
        ## array   
        res = []
        for xi in x:
            h += self.i_h(xi)
            ## for n inputs create n outputs, so we predict after every word
            h = self.bn(F.relu(self.h_h(h)))
            res.append(self.h_o(h))
        return torch.stack(res)

In [64]:
learn = Learner(data, Model2(), metrics=accuracy)

## note the accuracy goes down, because we only have one word as state to use 3 word 2 to use

In [65]:
learn.fit_one_cycle(10, 1e-4, pct_start=0.1)

epoch,train_loss,valid_loss,accuracy


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation

## Maintain state

In [53]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        ## this changed compared to above: we keep the state and do not just always throw it away
        self.h = torch.zeros(x.shape[1], nh).cuda()
        ## our accuracy goes up, see belo
        
    def forward(self, x):
        res = []
        h = self.h
        for xi in x:
            h = h + self.i_h(xi)
            h = F.relu(self.h_h(h))
            res.append(h)
        self.h = h.detach()
        res = torch.stack(res)
        res = self.h_o(self.bn(res))
        return res

In [54]:
learn = Learner(data, Model3(), metrics=accuracy)

In [55]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy
1,3.522065,3.397634,0.132812
2,3.160626,2.765389,0.384972
3,2.605956,2.176800,0.466120
4,2.094769,1.952014,0.430490
5,1.733199,1.858698,0.446782
6,1.485338,1.811119,0.434251
7,1.298967,1.859980,0.490254
8,1.144583,1.988956,0.466352
9,1.029006,2.114419,0.456064
10,0.924533,2.132976,0.503330


## nn.RNN

2:15:05 https://www.youtube.com/watch?v=9shn3Oz7Ptw

Feed an RNN into an RNN

In [66]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh) ## equivalent to above loop
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(1, x.shape[1], nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [57]:
learn = Learner(data, Model4(), metrics=accuracy)

In [58]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy
1,3.491045,3.436497,0.249683
2,3.077066,2.712121,0.418527
3,2.514287,2.175257,0.460396
4,2.044790,2.043891,0.315207
5,1.712145,2.073456,0.353984
6,1.423902,2.157615,0.396504
7,1.173752,1.952995,0.442443
8,0.963730,1.980419,0.456683
9,0.802111,1.939671,0.478032
10,0.683895,1.834323,0.502243


## 2-layer GRU

These large networks get difficult to train. We could add skip connections e.g.
Often, for RNNs we use a little mini network to decide how much of the state we should be keeping. Those things are often called LSTM or GRUs, see 2nd part of fast.ai

In [59]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        ## same as above, only woth 2 layers
        self.rnn = nn.GRU(nh,nh,2)
        ## 2:18:45L throw this away and make document classification https://www.youtube.com/watch?v=9shn3Oz7Ptw
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [60]:
learn = Learner(data, Model5(), metrics=accuracy)

In [61]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,3.083793,2.684498,0.423499
2,2.146678,1.781405,0.503463
3,1.296604,2.440667,0.569496
4,0.655731,2.192931,0.676361
5,0.341344,2.323763,0.670439
6,0.182797,2.239986,0.709932
7,0.110086,2.394657,0.698981
8,0.063283,2.414212,0.700926
9,0.044585,2.398133,0.693147
10,0.029014,2.408566,0.692021


## fin

How to continue 2:19:09 https://www.youtube.com/watch?v=9shn3Oz7Ptw